In [1]:
import onnx
import onnxruntime as ort
import cv2
import numpy as np
from utils import flow_to_image

In [2]:
modelfile = '/home/ruihu/repo/ptlflow/fastflownet.onnx'
img1path = '/home/ruihu/repo/ptlflow/data/1080p_30fps_TransfigCourtyard0099.png'
img2path = '/home/ruihu/repo/ptlflow/data/1080p_30fps_TransfigCourtyard0100.png'

In [3]:
session = ort.InferenceSession(modelfile)

In [4]:
inputs = session.get_inputs()
inputs_names = [inputs[i].name for i in range(len(inputs))]
print(inputs_names) 
shape = inputs[0].shape
print(shape)

['onnx::Slice_0']
[1, 6, 768, 1280]


In [5]:
outputs = session.get_outputs()
outputs_names = [outputs[i].name for i in range(len(outputs))]
print(outputs_names)
outputs_shape = outputs[0].shape
print(outputs_shape)

['30999']
[1, 1, 2, 768, 1280]


In [6]:
img1 = cv2.imread(img1path)
print(img1.shape)
img2 = cv2.imread(img2path)
print(img2.shape)

(720, 1280, 3)
(720, 1280, 3)


In [7]:
img1 = cv2.resize(img1,(shape[3],shape[2])).astype(np.float32) / 255.0
img2 = cv2.resize(img2,(shape[3],shape[2])).astype(np.float32) / 255.0
imgs = np.concatenate([img1,img2],axis=-1)
imgs = imgs.transpose(2,0,1)
imgs = np.expand_dims(imgs,axis=0)
imgs.shape

(1, 6, 768, 1280)

In [8]:
output = session.run(outputs_names,{inputs_names[0]:imgs})

In [9]:
flow = output[0][0][0]
flow = flow.transpose(1,2,0)
flow = flow_to_image(flow)
flow.shape

(768, 1280, 2)

In [11]:
cv2.namedWindow('Estimated Flow',cv2.WINDOW_NORMAL)
cv2.imshow('Estimated Flow', flow)
cv2.waitKey(0)

error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<3, 4>; VDcn = cv::impl::{anonymous}::Set<3, 4>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 2


: 